# TEMPORAL ANOMALIES FOR ANDES FIRES PIPELINE

*   Import Libraries -> Check disponibilities of the libraries
*   Import Datasets -> Take dataset from GEE Catalog
*   Final Code
  1. Preprocesing MCD14ML CSV -> formatting of files directly downloaded from the NASA FIRMS server.
  2. Load SHAPEFILE like a dataset in GEE -> Necessary to access files via GEE.
  3. Preprocessing MCD14ML GEE -> We extract ground cover type and elevation information through the locations of the actives fires.
  4. Surfaces Extraction WorldCover in GEE -> we calculate the total area of land cover in our study area.
  5. Data Filtering Pre-Qgis -> we narrow down the data by selecting only the land cover types of interest.
  6. Use of ST-DBSCAN Cluster Spatio-Temporal in Qgis -> use the ST-DBSCAN spatio-temporal clustering algorithm in Qgis to cluster fires presumed to belong to the same source.
  7. Data Filtering Post-Qgis -> Cleaning of the data after using the ST-DBSCAB algorithm, keeping only the first elements of each cluster, assuming these as the respective origins.
  8. Soil Surface -> Calculation of the total areas associated with each detected active fire

# **Import Libraries**

In [ ]:
import os
from pathlib import Path
import glob
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
from tqdm.notebook import tqdm
import ee
import geemap
from datetime import datetime

#Compte Earth Engine
ee.Authenticate()
ee.Initialize(project='ee-villa45ramos')


# **Import Dataset**

In [ ]:
datasets= {'AndesTropi':ee.FeatureCollection("projects/ee-villa45ramos/assets/AndesTropi2"),
           'dem30m': ee.Image("USGS/SRTMGL1_003"),
           'WorldCoverESA':ee.ImageCollection('ESA/WorldCover/v100'),
           'datacountries':ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level0"),
           'andes':ee.Geometry.BBox( -84.0, -30.0, -62.0, 12.0).buffer(1000)
}

# **Final Code**

## *1. Preprocessing MCD14ML CSV Format*

In [ ]:
# -*- coding: utf-8 -*-
"""
Modified on 16/07/2024 14:38:22
Version 4.1.0
@author: jvilla
"""
def merge_csv(files_pattern,output_name):
  """
  This function takes a list of files and merge in only one CSV.

  Parameters:
  -------------
  files_pattern: str
    Glob pattern to match files to merge.
  output_name: str
    Name of the output CSV file.

  Returns:
  -------------
  all_csv: pd.DataFrame
    Merged DataFrame.
  """
  try:
    # Get a list of files matching the glob pattern
    list_files = glob.glob(files_pattern)

    # Read each CSV file into a DataFrame and concatenate them
    all_csv = pd.concat(map(pd.read_csv, list_files), ignore_index=True)

    # Save the merged DataFrame to a CSV file
    all_csv.to_csv(output_name, index=False, encoding='utf-8-sig')

    return all_csv
  except Exception as e:
    print(f"An error occurred: {e}")

def filt_csv(file_path):
  """
  This function takes a CSV file and filter by confidence (>80% High Level Confidence), type of thermal anomalie detected
  (0 = presumed vegetation fire, 2 =other static land source), select only the columns needed, add a new column with the name of the country,
  and add a new column with the name of the climate zone.

  Parameters:
  -------------
  file_path: str
      Path to the CSV file to filter.

  Returns:
  -------------
  filtered_df: pd.DataFrame
    Filtered DataFrame.

  """
  try:

    #Get the file path
    file_name = Path(file_path).stem

    #Read the CSV file
    df = pd.read_csv(file_path)

    #Filter par confidence des anomalies thermiques supérieur à 80%
    filtered_df=df[df['confidence'] >= 80]

    #Selection des types de anomalies 0:suspected vegetation fire et 2:other static terrestrial source
    filtered_df=filtered_df[(filtered_df['type'] == 0) | (filtered_df['type'] == 2)]

    #Selection des colonnes d'interet
    filtered_df = filtered_df.iloc[:, [0,1,5,6,7,9,11,12]]

    #Add country name
    filtered_df["country"]=file_name

    #Add climate zone
    conditionlist = [
      (filtered_df["latitude"] >= -5),
      (filtered_df["latitude"] >= -8) & (filtered_df["latitude"] < -5),
      (filtered_df["latitude"] <= -8),
    ]
    choicelist = ["Zone_Equatorial", "Transition_Zone", "South_Zone"]
    filtered_df["Zone_Clima"] = np.select(conditionlist, choicelist, default="Not Specified") #Add climate zone

    return filtered_df

  except Exception as e:
    print(f"An error occurred: {e}")

def csv_to_shp(df,output_dir):
  """
  This function takes the coordinates (longitude,latitude) of every point and coverts into a shapefile
  with a projection EPSG:4326.

  Parameters:
  -------------
  df: pd.DataFrame
    DataFrame with coordinates.
  output_dir: str
    Directory of the output shapefile.

  Returns:
  -------------
  gdf: gpd.GeoDataFrame
    GeoDataFrame with the coordinates converted to a shapefile.

  """
  try:
    #Conversion of coordinates into Point objet
    df['coordinates'] = df[['longitude', 'latitude']].apply(Point, axis=1)

    #Creation de GeoDataframe
    gdf = gpd.GeoDataFrame(df, geometry = 'coordinates')

    #Set CRS
    gdf = gdf.set_crs('EPSG:4326')

    #Export to shapefile
    gdf = gdf.to_file(output_dir)
    return gdf

  except Exception as e:
    print(f"An error occurred: {e}")

def fusdatacsv(file_dict):
  """
  This function takes files with the word 'modis*.csv', merge in only one CSV and convert it into a shapefile

  Parameters:
  -------------
  file_dict: dict
    Dictionary with input and output paths for CSV and shapefiles.

  Returns:
  -------------
    None
  """
  n=len(file_dict)
  for k,i in zip(file_dict.keys(),range(1,len(file_dict)+1)):
    merge_csv(os.path.join(file_dict[k][0],'modis*.csv'),file_dict[k][1])
    print(f"Merge csv Terminée étape {i}/7")
    csv_to_shp(file_dict(file_dict[k][1]),file_dict[k][2][:-3]+'shp')
    print(f"CSV to SHP filt Terminée {i}/7")

def mergShape(shapefile_dir, output_shapefile):
  """
  This function takes a list of shapefiles and merge in only one shapefile

  Parameters:
  -------------
  shapefile_dir: str
    Directory containing shapefiles to merge.
  output_shapefile: str
    Path for the output merged shapefile.

  Returns:
  -------------
  AndesTropi : gpd.GeoDataFrame
    Merged GeoDataFrame.
  """
  try:
    # Get a list of shapefiles in the directory
    list_shapes = glob.glob(os.path.join(shapefile_dir,'*.shp'))

    # Read each shapefile into a GeoDataFrame and concatenate them
    GDF_list= list()
    GDF_list = [gpd.read_file(list_shapes[i]) for i in tqdm(range(len(list_shapes)))]

    # Merge the GeoDataFrames
    AndesTropi = gpd.GeoDataFrame(pd.concat(GDF_list,ignore_index=True), crs=GDF_list[0].crs)

    # Save the merged GeoDataFrame to a shapefile
    AndesTropi = AndesTropi.to_file(output_shapefile)

    return AndesTropi

  except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
dir_1='/content/drive/MyDrive/PFE_CIRAD_AMAP/'
dir_2='/content/drive/MyDrive/PFE_CIRAD_AMAP/Data_pays/'
dir_3='/content/drive/MyDrive/PFE_CIRAD_AMAP/shape_pays/'
dir_SHPf='/content/drive/MyDrive/PFE_CIRAD_AMAP/shape_pays/AndesTropiv3.shp'
pays=["Argentina","Bolivia","Chile","Colombia","Ecuador","Peru","Venezuela"]
datacsv= {f"csv{i}": [f'{dir_1}{pays[i]}', f'{dir_2}{pays[i]}.csv',f'{dir_3}{pays[i]}.shp'] for i in range(len(pays))}
fusdatacsv(datacsv)
mergShape(dir_3,dir_SHPf)
print("Processus Pretraitement MCD14ML CSV Format Terminée")

## *2. Load SHAPEFILE like a dataset in GEE*

-> *Necessary to continue processes within GEE*





## *3. Preprocessing MCD14ML in GEE*

In [ ]:
# -*- coding: utf-8 -*-
"""
Modified on 16/07/2024 14:38:50
Version 2.2.0
@author: jvilla
"""
def AnomaliesThermiques(roi,dem,wc,asset):
  def get_worldcover(roi,wc):
    """
    This function get the mosaic of WorldCover within the region of interest (roi).

    Parameters:
    -------------
    roi: ee.Geometry
      Region of interest (geometry).
    wc: ee.ImageCollection
      WorldCover image collection.

    Returns:
    -------------
    wc_mosaic: ee.Image
      Mosaic of WorldCover within the region of interest.
    """
    #Function to clip image by ROI
    def clip_study_area(img):
      return img.clip(roi)

    #Filter WorldCover image by bound of ROI, then Clip image and convert in one mosaic image
    wc_mosaic=wc.filterBounds(roi).map(clip_study_area).mosaic()

    return wc_mosaic

  def extract_fire_activities(wc_mosaic,dem,roi,asset):
    """
    Extract fire activities from the asset, adding WorldCover and elevation data.

    Parameters:
    -------------
    wc_mosaic: ee.Image
      Mosaic of WorldCover within the region of interest.
    dem: ee.Image
      Elevation data.
    roi: ee.Geometry
      Region of interest (geometry).
    asset: ee.FeatureCollection
      MCD14ML asset.

    Returns:
    -------------
    fire_activities: ee.FeatureCollection
      Fire activities within the region of interest.
    """
    #Function to format date
    def format_date(fc):
      return fc.set('ddmmyy', ee.Date(fc.get('acq_date')).format('ddMMyy'))

    #Function to add WorldCover and elevation data to feature
    def add_cover_elevation(feat):
      geom = feat.geometry()

      #Use of mode because discrete data in a buffer of 1000m (MODIS resolution)
      wcover = wc_mosaic.reduceRegion(
          ee.Reducer.mode(), feat.geometry(), 1000).get('Map')

      #Use of mean because of continous data of elevation inside the buffer of 1000m (MODIS resolution)
      elev30 = dem.clip(roi).reduceRegion(
          ee.Reducer.median(), feat.geometry(), 1000).get('elevation')
      return ee.Feature(geom).copyProperties(feat).set({
          'wc': wcover,
          'elev30':elev30
      })

    #Filter elevation greater than 2000 (definition of Tropical Andes)
    filtered_asset  = asset.filterBounds(wc_mosaic.geometry()).map(format_date).map(add_cover_elevation, True).filterMetadata('elev30', "greater_than", 2000)
    return filtered_asset

  wc_mosaic = get_worldcover(roi, wc)
  fire_activities = extract_fire_activities(wc_mosaic, dem, roi, asset)
  return fire_activities

MODWorldCoverv = AnomaliesThermiques(datasets["andes"],datasets["dem30m"],datasets["WorldCoverESA"],ee.FeatureCollection("projects/ee-villa45ramos/assets/AndesTropiv3"))
print("Processus FireActivity terminée")

In [ ]:
task = ee.batch.Export.table.toDrive(
    collection=MODWorldCoverv,
    description='MODWorldCoverv5',
    folder='Extra_data',
    fileFormat='CSV')

task.start()

print("Processus Pre-traitement MCD14ML in GEE terminée")

Processus Pretraitement MCD14ML in GEE terminée


## *4. Surfaces extraction WorldCover in GEE*

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on 16/07/2024 14:45:16
Version 4.1.0
@author: jvilla
"""
def Cover_surface(codesoil,pays,cover,dem,AOI):
  """
  This function takes extract the total surface of soil in the region of interest (AOI)

  Parameters:
  -------------
  codesoil: int
    Code of soil class.
  pays: ee.FeatureCollection
    List of countries.
  cover: ee.ImageCollection
    WorldCover image collection.
  dem: ee.Image
    DEM Elevation data.
  AOI: ee.Geometry
    Region of interest (geometry).

  Returns:
  -------------
  Soil_cover: ee.FeatureCollection
    Total surface of soil in the region of interest per Soil class.
  """
  #Function to select only countries with presence in the Tropical Andes.
  def country(pays):

    #Countries codes for-> 195:Peru, 51:Chile, 12:Argentina, 33:Bolivia, 73:Ecuador, 57:Colombia, 263:Venezuela
    countries = ee.List([195,51,12,33,73,57,263]);
    countriesshp = pays.filter(ee.Filter.inList('ADM0_CODE',countries))
    filterc = ee.Filter.inList('ADM0_CODE',countries)
    countriesshp = pays.filter(filterc)
    def remove_non_polygons(geometry):
      return ee.Algorithms.If(ee.Algorithms.IsEqual(ee.Geometry(geometry).type()
                            , ee.String('Polygon'))
                            , ee.Feature(ee.Geometry(geometry)), None)
    def clean_polygon_feature(feature):
      geometries = feature.geometry().geometries();
      geometries_cleaned = ee.FeatureCollection(geometries.map(remove_non_polygons)).union().first();
      return ee.Feature(feature).setGeometry(geometries_cleaned.geometry())
    return countriesshp.map(clean_polygon_feature)

  #Function to clip WorldCover Raster by ROI and make a mosaic image
  def worldcover(cover,AOI):
    def clipStudyArea(img):
      return img.clip(AOI)
    return cover.filterBounds(AOI).map(clipStudyArea).mosaic()

  #Function to get the total surface of soil in the region of interest
  def mask_soil(dem,AOI,cover,pays):
    dem2 = dem.clip(AOI)
    elev = dem2.gte(2000)
    soil = worldcover(cover,AOI).updateMask(elev)

    soilCover  = ee.Image(1).updateMask(soil.select('Map').eq(codesoil))#Grasland #Change of mask to updateMask
    area_pxa_soilCover_countries = soilCover.multiply(ee.Image.pixelArea()).reduceRegions(
                                                              collection = country(pays),
                                                              reducer = ee.Reducer.sum(),
                                                              scale = 10)
    #Function to convert area to hectares
    def getArea(feature):
      area = ee.Number(feature.get('sum')).divide(1e6)
      return feature.set('area', area)
    return area_pxa_soilCover_countries.map(getArea).select('area','ADM0_NAME')
  Soil_cover = mask_soil(dem,AOI,cover,pays)
  return Soil_cover


Processus FireActivity terminée


In [ ]:
codesoil=40 #Changer code soil-> 10:TreeCover, 20:Shrubland, 30: Grassland, 40 Cropland

grid = geemap.fishnet(ee.Geometry.BBox( -84.0, -30.0, -62.0, 12.0).buffer(1000), h_interval=4.5, v_interval=6.1, delta=1)
gridSize=grid.size().getInfo();
gridList=grid.toList(gridSize);

for i in tqdm(range(gridSize)):
  AOI = ee.Feature(gridList.get(i)).geometry()
  MOD_soil_Area = Cover_surface(codesoil,datasets["datacountries"],datasets["WorldCoverESA"],datasets["dem30m"],datasets["andes"])
  task = ee.batch.Export.table.toDrive(
    collection=MOD_soil_Area,
    description=f'Cropland_Area_part{i}', #Changer name soil-> 10:TreeCover, 20:Shrubland, 30: Grassland, 40 Cropland
    folder='Inter_data_ThermalAnomalies',
    fileFormat='CSV')
  task.start()
  print(f"Execution part: {i}")
print("Processus Extraction surfaces WordlCover in GEE terminée")

## *5. Data Filtering Pre-Qgis*

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on 11/07/2024 17 17:31:36
Version 3.0.0
@author: jvilla
"""
def filtZonesSeches(dir_csv,dir_out):
	"""
	This function takes a CSV file and filters by soil coverage, reduces columns and rename columns/elements.

	Parameters:
	-------------
	dir_csv: str
		Path to the CSV file to filter.
	dir_out: str
		Path to the output CSV file.

	Returns:
	-------------
	df: pd.DataFrame
		Filtered DataFrame.
	"""
	try:
		# Read the CSV file
		df = pd.read_csv(dir_csv)

		# Filter by soil coverage 10:Tree cover, 20:Shrubland, 30:Grassland and 40:Cropland
		df = df[df['wc'] <= 40]

		# Reduce columns and rename elements
		df['month'] = pd.DatetimeIndex(df['acq_date']).month
		df['year'] = pd.DatetimeIndex(df['acq_date']).year
		new_df=df.iloc[:,[1,2,6,8,9,13,12,15,16,10,11]]
		new_df.columns=['Zone_Climatique','Date_Acquisition','Pays','Elevation(m)','FRP','WordlCover','Satellite','Mois','Annee','Lat(°)','Long(°)']
		new_df['WordlCover']=new_df['WordlCover'].replace([10, 20, 30, 40], ["Tree cover", "Shrubland", "Grassland", "Cropland"])

		# Write the filtered DataFrame to a CSV file
		new_df.to_csv(dir_out, index=False)
		print("Processus Nettoyage des données MCD14ML en Pandas terminée")

		return new_df
	except Exception as e:
		print(f"An error occurred: {e}")

In [ ]:
filtZonesSeches("/content/drive/MyDrive/ content drive MyDrive PFE_CIRAD_AMAP/MODWorldCoverv5.csv","/content/drive/MyDrive/PFE_CIRAD_AMAP/1_MCD14ML/Inter_data_ThermalAnomalies/MODWorldCover_fil.csv")

Processus Nettoyage des données MCD14ML en Pandas terminée


## *6. Use of ST-DBSCAN Cluster Spatio-Temporal in Qgis*

## *7. Data Filtering Post-Qgis*

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on 16/07/2024 14:05:23
Version 2.1.0
@author: jvilla
"""
def filClusters(dir_csv,dir_out):
	"""
	This function takes a CSV file and filters by cluster ID.

	Parameters:
	-------------
	dir_csv: str
		Path to the CSV file to filter.
	dir_out: str
		Path to the output CSV file.

	Returns:
	-------------
	df: pd.DataFrame
		Filtered DataFrame.
	"""
	try:
		# Read the CSV file
		df=pd.read_csv(dir_csv)

		# Separate rows with and without cluster IDs
		df_slice1=df[df['CLUSTER_ID'].isna()]#Independent Fires
		df_slice2=df[~df['CLUSTER_ID'].isna()]#Interdependent Fires

		# Sort and remove duplicates
		df_slice2=df_slice2.sort_values(by=['Date_Acquisition'])
		df_slice2= df_slice2.drop_duplicates(subset=["CLUSTER_ID"],keep='first')

		# Combine the slices back together
		df=pd.concat([df_slice1,df_slice2],axis=0,ignore_index=True)

		# Ensure only the first 11 columns are included in the output
		df=df.iloc[:,:11]

		# Write the filtered DataFrame to a CSV file
		df.to_csv(dir_out, index=False)

		print("Processus Nettoyage des données MCD14ML en Post Clustering terminée")

		return df


In [ ]:
filClusters('/content/drive/MyDrive/PFE_CIRAD_AMAP/1_MCD14ML/Inter_data_ThermalAnomalies/MODWorldCover_fil_stdscan15j.csv','/content/drive/MyDrive/PFE_CIRAD_AMAP/1_MCD14ML/Outputs_ThermalAnomalies/MCD14ML_TA.csv')

Processus Nettoyage des données MCD14ML en Post Clustering terminée


## **8. Soil Surface** #

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on 16/07/2024 15:40:41
Version 1.1.0
@author: jvilla
"""
def merge_csv_SurfaceSoil(dir,pays):
  """
  This function merge csv files into one and summarize by Sum of area

  Parameters:
  -------------
  dir: str
    Directory of the input CSV files.
  pays: list
    List of countries.

	Returns:
	-------------
  ALL_csv: pd.DataFrame
    Merged DataFrame.
  """

  for i in tqdm(range(0,len(pays))):
    name=pays[i]
    path_fil=os.path.join(dir,f'*{name}*.csv')
    list_files = glob.glob(path_fil)
    All_csv = pd.concat(map(pd.read_csv, list_files), ignore_index=True)
    All_csv = All_csv.groupby(['ADM0_NAME'])['area'].sum().reset_index()
    All_csv.to_csv(f'{dir}/{name}_all.csv', index=False, encoding='utf-8-sig')#testee dans le code integree
    print(f"Merge csv Terminée étape {i}/3")
    return ALL_csv

In [ ]:
dir_1='/content/drive/MyDrive/PFE_CIRAD_AMAP/1_MCD14ML/Inter_data_ThermalAnomalies'
pays=["TreeCover_Area_part","Shrubland_Area_part","Grassland_Area_part","Cropland_Area_part"]

merge_csv_SurfaceSoil(dir_1,pays)

print("Processus Surface Soil WordlCover CSV Format Terminée")

  0%|          | 0/4 [00:00<?, ?it/s]

Merge csv Terminée étape 0/4
Merge csv Terminée étape 1/4
Merge csv Terminée étape 2/4
Merge csv Terminée étape 3/4
Processus Surface Soil WordlCover CSV Format Terminée


In [ ]:
dtf=pd.read_csv('/content/drive/MyDrive/PFE_CIRAD_AMAP/1_MCD14ML/Inter_data_ThermalAnomalies/Grassland_Area_part_all.csv')
Grassland_area=dtf.groupby(['ADM0_NAME'])['area'].sum().reset_index()
#Grassland_area.head()
Grassland_area.to_csv('/content/drive/MyDrive/PFE_CIRAD_AMAP/1_MCD14ML/Inter_data_ThermalAnomalies/Grassland_Area_part_all2.csv', index=False, encoding='utf-8-sig')